## This code collects potential terms and targets given a set of seed tweets.
## We first use KeyBERT to get key terms. Then use dictionaries to filter out known words.

In [ ]:
!pip install keybert

In [ ]:
!pip install pyenchant==3.0.0a1

In [ ]:
!apt-get install enchant-2

In [7]:
import pandas as pd
data = pd.read_csv("<-read the seed dataset->")

In [2]:
from keybert import KeyBERT
kw_model = KeyBERT()

def get_key_terms(text):
  keywords = kw_model.extract_keywords(text)
  terms = ''
  for word, conf in keywords:
    if conf > 0.5:
      terms = terms + ' ' + word
  return terms

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
# get the data that are hateful
data_hate = data[data['ground_truth']==1]

In [9]:
data_age = data_hate[data_hate['category']=='ageism']
data_asian = data_hate[data_hate['category']=='asian']
data_vac = data_hate[data_hate['category']=='vaccine']
data_mask = data_hate[data_hate['category']=='mask']
data_us = data_hate[data_hate['category']=='us_capitol']
data_rus = data_hate[data_hate['category']=='rus_ukr']

In [25]:
def is_english_word(word):
    d = enchant.Dict("en_US")
    return d.check(word), d.suggest(word)

In [65]:
def is_urban_word(word, date_key):
    url = f"https://api.urbandictionary.com/v0/define?term={word}"
    response = requests.get(url)
    data = response.json()
    if data['list']:
        when = data['list'][0]['written_on']

        # check if the word is a new term or not
        if datetime.strptime(when, "%Y-%m-%dT%H:%M:%S.%fZ") < date_key:
            return False
        else:
            return True
    else:
        return True
    time.sleep(1)

In [77]:
import random
import enchant

def keyword_extract(data, month, year):
  # get keywords
  terms = set()
  for i in range(5):
    random_number = random.randint(10, 20)
    try:
      data_seed = data.sample(n = random_number)
    except:
      continue

    for date, text, gt in zip(data_seed['month'].tolist(), data_seed['original_text'].tolist(), data_seed['ground_truth'].tolist()):
      if date == month:
        if gt == 1:
          keywords_text = get_key_terms(text)
          for word in keywords_text.split():
            terms.add(word)
  terms = list(terms)
  print("\nKeywords found:\n", terms)

  new_derog_terms = set()
  for word in terms:
    if word.isnumeric():
      continue

    cleaned_word = ''.join(letter for letter in word if letter.isalnum())
    if cleaned_word != '':
      present, suggs = is_english_word(cleaned_word)
    else:
      present = True
    if not present:
        new_derog_terms.add(cleaned_word)
  new_derog_terms = list(new_derog_terms)
  # print("\n",new_derog_terms)

  final_set = set()
  for word in new_derog_terms:
      if is_urban_word(word, datetime.strptime(f'{year}-{month}-1', '%Y-%m-%d')):
          final_set.add(word)
  final_terms = list(final_set)
  print("The final terms are:", final_set)
  return final_terms

In [78]:
keyword_extract(data_age, 1, 2020)
keyword_extract(data_age, 4, 2020)
keyword_extract(data_age, 7, 2020)
keyword_extract(data_age, 10, 2020)


Keywords found:
 ['becky', 'boomers', 'beckydonthavethegoodhairthough']
The final terms are: {'beckydonthavethegoodhairthough'}

Keywords found:
 ['ageism', 'boomers']
The final terms are: set()

Keywords found:
 ['mustard', 'covididiot']
The final terms are: {'covididiot'}

Keywords found:
 ['boomers', 'boomer', 'remover']
The final terms are: set()


[]

In [80]:
keyword_extract(data_asian, 1, 2020)
keyword_extract(data_asian, 4, 2020)
keyword_extract(data_asian, 7, 2020)
keyword_extract(data_asian, 10, 2020)


Keywords found:
 ['chinese', 'kung', 'fuckchina', 'coronaravirus', 'plague']
The final terms are: {'coronaravirus', 'fuckchina'}

Keywords found:
 ['hongkongers', 'unemployed', 'chinesevirus', 'malaysia', 'fuckchina', 'china_is_terrorist', 'hongkong']
The final terms are: {'chinesevirus', 'chinaisterrorist', 'fuckchina'}

Keywords found:
 ['chinaliedpeopledied', 'chinavirus', 'chineseappsblocked', 'covid', 'chinagetoutofiran', 'coronavirus', 'pandemic', 'chinafreeworld']
The final terms are: {'chinaliedpeopledied', 'chinavirus', 'chineseappsblocked', 'covid', 'chinagetoutofiran', 'chinafreeworld'}

Keywords found:
 ['chinesevirus', 'wuhanvirus', 'chinavirus', 'ccp_is_terrorist']
The final terms are: {'chinesevirus', 'wuhanvirus', 'chinavirus', 'ccpisterrorist'}


['chinesevirus', 'wuhanvirus', 'chinavirus', 'ccpisterrorist']

In [81]:
keyword_extract(data_vac, 1, 2020)
keyword_extract(data_vac, 4, 2020)
keyword_extract(data_vac, 7, 2020)
keyword_extract(data_vac, 10, 2020)


Keywords found:
 ['vaxers', 'antivax']
The final terms are: {'vaxers', 'antivax'}

Keywords found:
 ['covidiots', 'contagious', 'coronavirusliar', 'vaccine', 'covid19']
The final terms are: {'coronavirusliar', 'covid19'}

Keywords found:
 ['covidiots', 'obv']
The final terms are: set()

Keywords found:
 ['covidiots', 'trumpvirus', 'covid', 'vaccine', 'vaccines']
The final terms are: {'covid'}


['covid']

In [83]:
keyword_extract(data_mask, 1, 2020)
keyword_extract(data_mask, 4, 2020)
keyword_extract(data_mask, 7, 2020)
keyword_extract(data_mask, 10, 2020)


Keywords found:
 []
The final terms are: set()

Keywords found:
 ['pence', 'maskhole']
The final terms are: {'maskhole'}

Keywords found:
 ['masks', 'masksdontwork', 'masksoffamerica', 'mask', 'louisianagov', 'independenceday', 'danny', 'dumptrump2020', 'maskhole']
The final terms are: {'masksdontwork', 'masksoffamerica', 'louisianagov', 'independenceday', 'dumptrump2020', 'maskhole'}

Keywords found:
 ['schumerresign', 'hole', 'masks', 'mask', 'orifice', 'schumer', 'sunscreen', 'votejoebiden', 'saban', 'maskhole']
The final terms are: {'schumerresign', 'votejoebiden', 'maskhole'}


['schumerresign', 'votejoebiden', 'maskhole']

In [84]:
keyword_extract(data_us, 1, 2021)
keyword_extract(data_us, 4, 2021)
keyword_extract(data_us, 7, 2021)
keyword_extract(data_us, 10, 2021)


Keywords found:
 ['cruz', 'qnuts', 'magamorons', 'magaterrorism', 'pence', 'lockthemup', 'magats', 'petition', 'impeached', 'wig', 'fuhrer', 'resign', 'kosher']
The final terms are: {'qnuts', 'magamorons', 'magaterrorism', 'lockthemup'}

Keywords found:
 ['stallone']
The final terms are: set()

Keywords found:
 ['magamorons', 'threattodemocracy', 'smart']
The final terms are: {'magamorons', 'threattodemocracy'}

Keywords found:
 ['lockthemup', 'teabagterrorists', 'loyalists', 'arizonapolitics', 'trumpsters', 'bannon', 'idiots']
The final terms are: {'lockthemup', 'arizonapolitics', 'teabagterrorists'}


['lockthemup', 'arizonapolitics', 'teabagterrorists']

In [85]:
keyword_extract(data_rus, 1, 2022)
keyword_extract(data_rus, 4, 2022)
keyword_extract(data_rus, 7, 2022)
keyword_extract(data_rus, 10, 2022)


Keywords found:
 []
The final terms are: set()

Keywords found:
 ['nazirussia', 'dmitri', 'ukrainians', 'russiawarcrimes', 'ukrainian', 'nazirussianarmy', 'genocideofukrainians']
The final terms are: {'genocideofukrainians', 'russiawarcrimes', 'nazirussia', 'nazirussianarmy'}

Keywords found:
 ['putin', 'russians', 'ukrainewar', 'russiaisaterroriststate', 'ukraineunderatta', 'sanctions', 'ukrainerussiawar', 'russianarmy', 'russianwarcrimes', 'russiaisaterrorisstate', 'russianfascism', 'putinisawarcriminal', 'genocideofukrainians', 'ukrainekrieg']
The final terms are: {'ukrainewar', 'russiaisaterroriststate', 'ukraineunderatta', 'putinisawarcriminal', 'ukrainerussiawar', 'russianarmy', 'russianwarcrimes', 'russiaisaterrorisstate', 'russianfascism', 'genocideofukrainians', 'ukrainekrieg'}

Keywords found:
 ['russiaisaterroriststate', 'zaporizhzhia', 'ruzzian', 'russiaisanazistate', 'putinists', 'kakhovka', 'russianlooters', 'ruscist']
The final terms are: {'russiaisaterroriststate', 'za

['russiaisaterroriststate',
 'zaporizhzhia',
 'ruzzian',
 'russiaisanazistate',
 'kakhovka',
 'russianlooters',
 'ruscist']